In [4]:
!pip install requests
!pip install bs4


zsh:1: command not found: pip
zsh:1: command not found: pip
zsh:1: command not found: pip
zsh:1: command not found: pip
zsh:1: command not found: pip


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/IT집중교육1

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
user_agent = {'User-agent': 'Mozilla/5.0'}
links=[]
data_list=[]
first_row = ['name', 'platform', 'metascore', 'userscore', 'developer', 'genre']
def get_href():
  links = []
  for i in range(0, 10) :
    url = "https://www.metacritic.com/browse/games/score/metascore/year/all/filtered?year_selected=2021&distribution=&sort=desc&view=detailed&page="+str(i)
    html = requests.get(url, headers=user_agent)
    soup = BeautifulSoup(html.content, 'html.parser')
    td = soup.find_all('td', attrs = {'class':'clamp-summary-wrap'})
    for k in td:
      a = k.find("a", class_="title")['href']
      links.append("https://www.metacritic.com/"+a)
  return links
def get_data(links):
  #rank = 
  check = 0
  temp_list = []
  for link in links:
    temp_genre=''
    #rank += 1
    url = link
    html = requests.get(url, headers=user_agent)
    soup = BeautifulSoup(html.content, 'html.parser')
    try:
      name = soup.find("div", class_="product_title").find("h1").get_text()
      #print(name)
      try:
        platform = soup.find("span", class_="platform").get_text()
      except:
        platform = "tbd"
      # try:
      #   release_date = soup.find("li", class_="summary_detail release_data").find("span", class_='data').get_text()
      # except:
      #   release_date = "tbd"
      try:
        metascore = soup.find("div", class_="metascore_wrap highlight_metascore").find("span", itemprop="ratingValue").get_text()
      except:
        metascore = "tbd"
        userscore = soup.select('a.metascore_anchor')[0].text
        check = 1
      if check != 1:
        userscore = soup.select('a.metascore_anchor')[1].text
      try:
        developer = soup.find("li", class_="summary_detail developer").find("span", class_="data").get_text()
      except:
        developer ="tbd"
      genre = soup.find("li", class_="summary_detail product_genre").find_all("span", class_="data")
      for i in range(len(genre)):
        temp_genre += genre[i].get_text()
      temp_list.append([name, platform, metascore, userscore, developer, temp_genre])
    except:
      pass
    #temp_dict[str(rank)] = {'name':name, 'platform':platform, 'metascore':metascore, 'userscore':userscore,'developer':developer,'genre':temp_genre}
  return temp_list
def make_csv(data_list):
  with open('2021topgame.csv', 'w', encoding='utf-8', newline='') as file:
    csvfile = csv.writer(file)
    csvfile.writerow(first_row)
    for row in data_list:
      csvfile.writerow(row)
links = get_href()
temp = get_data(links)
data_list += temp
make_csv(data_list)

  









In [16]:
import os
from glob import glob
import pandas as pd

os.chdir('./')

file_names = glob('./*.csv')
total = pd.DataFrame()
for file_name in file_names:
    temp = pd.read_csv(file_name, encoding='utf-8')
    total=pd.concat([total, temp])
total.to_csv('./total.csv', index=False)
data = pd.read_csv('total.csv')
data[(data['metascore'] == 'tbd')] = 0
data[(data['userscore'] == '\ntbd\n')] = 0
data['metascore'] = data['metascore'].astype(int)
data['userscore'] = data['userscore'].astype(float)
data = data.sort_values('metascore', ascending=False)




,name,platform,metascore,userscore,developer,genre
19787,The Legend of Zelda: Ocarina of Time,\n ...,99,9.1,\nNintendo,Action AdventureFantasy
6831,Grand Theft Auto IV,\n\n ...,98,7.9,\nRockstar North,Action AdventureModernModernOpen-World
6631,Grand Theft Auto IV,\n\n ...,98,7.9,\nRockstar North,Action AdventureModernModernOpen-World
4485,Tony Hawk's Pro Skater 2,\n ...,98,7.4,\nNeversoft Entertainment,SportsAlternativeSkateboarding
6930,Grand Theft Auto IV,\n\n ...,98,7.8,\nRockstar North,Action AdventureModernModernOpen-World
...,...,...,...,...,...,...
13226,0,0,0,0.0,0,0
13225,0,0,0,0.0,0,0
13224,0,0,0,0.0,0,0
13223,0,0,0,0.0,0,0


In [17]:
data.head(5)

,name,platform,metascore,userscore,developer,genre
19787,The Legend of Zelda: Ocarina of Time,\n ...,99,9.1,\nNintendo,Action AdventureFantasy
6831,Grand Theft Auto IV,\n\n ...,98,7.9,\nRockstar North,Action AdventureModernModernOpen-World
6631,Grand Theft Auto IV,\n\n ...,98,7.9,\nRockstar North,Action AdventureModernModernOpen-World
4485,Tony Hawk's Pro Skater 2,\n ...,98,7.4,\nNeversoft Entertainment,SportsAlternativeSkateboarding
6930,Grand Theft Auto IV,\n\n ...,98,7.8,\nRockstar North,Action AdventureModernModernOpen-World
